# Analisi e visualizzazione dei dati del Lavoro in italia (1996-2018) (ISTAT)
Il progetto consiste nell'analisi dei dati presenti sul sito <a href="https://www.kaggle.com">Kaddle</a> e relativi alla condizione lavorativa in italia:
<a href="https://www.kaggle.com/mpwolke/cusersmarildownloadspopolazionecsv">Population by labour status - Italy</a>. <br>
La fonte dei dati è ISTAT e si tratta delle rilevazioni fatte dal 1996 al secondo trimestre 2019.<br>
Nello specifico l'attività condotta consiste in:
1. Caricamento e prima esplorazione del Dataset
2. Preparazione dei valori e data cleaning
3. Rimozione delle features ridondanti
4. Creazione di nuove feature e nuovi aggregati
5. Reshaping del Dataframe 
6. Visualizzazione interattiva con Plotly, Plotly Express e Plotly Dash

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

#imposto il template di default di Plotly Express
pio.templates.default = "seaborn"

# Verifico la presenza del notebook nella cartella

#import os
#for dirname, _, filenames in os.walk('.'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))


## Analisi esplorativa del DataFrame
Carico il file "popolazione.csv" usando l'encoding corretto e il separatore ";" e visualizzo le prime 5 righe del DataFrame.
Dalla primissima analisi emerge che il dataset è ordinato ma presenta una formattazione anomala nella feature "value" che dovrebbe essere espressa in migliaia (cfr. "tipo dato"), ma presenta delle stringhe contenenti il punto (".") con significato ambiguo: sia come separatore dei decimali che delle migliaia.

In [2]:
df = pd.read_csv('popolazione_orig.csv', encoding='cp1252', sep=';', decimal='.')
df.head(5)

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value
0,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996,1996,1.716.452
1,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q2,T2-1996,1.717.624
2,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q1,T1-1996,1.719.434
3,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q4,T4-1996,1713.29
4,ITC1,Piemonte,POP,popolazione 0 anni e più (in migliaia),2,femmine,Y15-74,15-74 anni,99,totale,TOT,totale,1996-Q3,T3-1996,1.715.458


A ulteriore verifica faccio un controlllo delle features numeriche e dei valori nulli:

In [3]:
df.describe()

,sexistat1
count,267234.000000
mean,4.000000
std,3.559033
min,1.000000
25%,1.000000
50%,2.000000
75%,9.000000
max,9.000000


In [4]:
df.isnull().sum()

itter107                            0
territorio                          0
tipo_dato_fol                       0
tipo dato                           0
sexistat1                           0
sesso                               0
eta1                                0
classe di età                       0
condizione_prof                     0
condizione professionale            0
condizione_prof_eu                  0
condizione professionale europea    0
time                                0
seleziona periodo                   0
value                               0
dtype: int64

Al momento l'unica feature numerica è relativa al codice Istat che rappresenta il sesso, non ci sono valori nulli.<br>
Verifico l'omgeneità rispetto alla feature "Territorio":

In [5]:
df['territorio'].value_counts()

Provincia Autonoma Trento             12147
Calabria                              12147
Sicilia                               12147
Friuli-Venezia Giulia                 12147
Puglia                                12147
Marche                                12147
Campania                              12147
Umbria                                12147
Molise                                12147
Provincia Autonoma Bolzano / Bozen    12147
Lombardia                             12147
Liguria                               12147
Emilia-Romagna                        12147
Piemonte                              12147
Basilicata                            12147
Lazio                                 12147
Veneto                                12147
Toscana                               12147
Sardegna                              12147
Valle d'Aosta / Vallée d'Aoste        12147
Trentino Alto Adige / Südtirol        12147
Abruzzo                               12147
Name: territorio, dtype: int64

Di seguito 20 righe della feature "value", che evidenziano il problema di formattazione:

In [6]:
df.value[20:40]

20    1.681.456
21    1.682.779
22       1684.7
23    1.677.971
24    1.680.374
25      1671.87
26    1.675.566
27    1.673.528
28    1.667.747
29    1.670.638
30    1.662.925
31      1663.52
32    1.664.856
33    1.661.069
34    1.662.256
35    1.661.071
36      1659.89
37    1.659.148
38    1.664.027
39    1.661.217
Name: value, dtype: object

<b>[-->]</b>La feature "Value" contiene valori disomogenei e non numeri decimali come ci si aspetterebbe (si tratta di valori espressi in migliaia), vi sono infatti valori espressi col primo punto come separatore delle migliaia e un secondo punto come separatore dei decimali.

### Esplorazione della condizione professionale
Inizio l'esplorazione delle diverse feature relative alla condizione professionale, per fare questo filtro i dati per regione, anno, classe di età e sesso al fine di ottenere una rilevazione significativa di tutte le rilevazioni relative alla condizione professionale. Il dataset infatti presenta 2 diverse classificazioni con relative codifiche che fanno riferimento alla "condizione professionale" e alla "condizione professionale europea".

In [7]:
df['condizione professionale'].astype('category').cat.categories

Index(['disoccupati', 'forze lavoro', 'inattivi',
       'non cercano e non disponibili a lavorare', 'occupati', 'totale',
       'zona grigia dell'inattività'],
      dtype='object')

In [8]:
df_once = df[(df.territorio == 'Sardegna') & (df['time'] == '2012') 
             & (df['classe di età'] == '15-64 anni') 
             & (df['sesso'] == 'totale')]
df_once

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value
89592,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,UNEM,disoccupati,2012,2012,106.792
107706,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,2,disoccupati,TOT,totale,2012,2012,106.792
125343,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,TOTIN,totale inattivi,2012,2012,432.788
160543,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,1_2,forze lavoro,TOT,totale,2012,2012,684.656
209975,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,TOT,totale,2012,2012,1.117.444
230765,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,LF,forze lavoro,2012,2012,684.656
232871,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3_4,inattivi,TOT,totale,2012,2012,432.788
234112,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3A_3B_3C,zona grigia dell'inattività,TOT,totale,2012,2012,121.814
243014,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3D,non cercano e non disponibili a lavorare,TOT,totale,2012,2012,310.974
245499,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,EMP,occupati,2012,2012,577.864


In [9]:
df_once = df[(df.territorio == 'Sardegna') & (df['time'] == '2012') 
           & (df['classe di età'] == '15-64 anni') 
           & (df['sesso'] == 'totale') & (df['condizione_prof_eu'] == 'TOT')]
df_once

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value
107706,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,2,disoccupati,TOT,totale,2012,2012,106.792
160543,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,1_2,forze lavoro,TOT,totale,2012,2012,684.656
209975,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,99,totale,TOT,totale,2012,2012,1.117.444
232871,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3_4,inattivi,TOT,totale,2012,2012,432.788
234112,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3A_3B_3C,zona grigia dell'inattività,TOT,totale,2012,2012,121.814
243014,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,3D,non cercano e non disponibili a lavorare,TOT,totale,2012,2012,310.974
267080,ITG2,Sardegna,POP,popolazione 0 anni e più (in migliaia),9,totale,Y15-64,15-64 anni,1,occupati,TOT,totale,2012,2012,577.864


Dall'esplorazione appena riportata risulta:
1. è necessario eliminare tutte le osservazioni con condizione_prof_eu != 'TOT' in quanto rindondanti
2. Condizione professionale = 'totale' --> Totale della popolazione nella fascia di età selezionata
3. Condizione professionale = 'forze lavoro' --> Popolazione - Inattivi
4. Condizione professionale = 'occupati' --> Forza lavoro - disoccupati
5. Condizione professionale = 'inattivi' --> Zona Grigia + Non cercano e non sono disponibili a lavorare

## Preparazione dei valori
### Preparazione dei valori (value)
Come evidenziato nella prima fase esplorativa, occorre mettere in qualità i valori numerici della feature "value" che al momento non sono utilizzabili. La strategia consiste nell'individuare il primo punto andando a tracciarne la posizione se la lunghezza della feature "value" è maggiore di 6, saranno infatti tracciate le posizioni del primo punto nei casi in cui il valore sia espresso nella forma:<br>
"x.xxx.xxx","x.xxx.xx", "x.xxx.x", "xxx.xxx", "xx.xxx.x" ...<br>
Nel caso la lunghezza della stringa sia 6 o meno i vaolri avranno la forma "xx.xxx", "x.xxx","xxx.xx", ...: in tutti questi casi la posizione del primo punto viene impostata a "0". 

In [10]:
#cerca l'indice del primo punto 
df['dot_index'] = np.where(df.value.str.len() > 6, df['value'].str.find('.'),0)
#Se la posizione del punto primo punto è 1 (per stringhe iniziali con lunghezza 
#iniziale > 6), procedo a rimuovere il primo punto. 
df['new_value'] = np.where(np.logical_and(df.dot_index < 2, df.dot_index > 0), 
                           df['value'].str.slice(stop=1) + df['value'].str.slice(start=2) ,df['value'])
df.new_value.head(20)

0     1716.452
1     1717.624
2     1719.434
3      1713.29
4     1715.458
5     1708.008
6     1711.124
7     1709.324
8     1704.614
9      1706.97
10    1699.068
11    1700.272
12     1702.26
13    1695.736
14    1698.004
15    1690.328
16    1693.467
17     1691.57
18    1686.992
19    1689.282
Name: new_value, dtype: object

Ora sovrascrivo la feature "value" con i valori ripuliti e verifico che i valori siano ora utilizzabili.

In [11]:
df['value'] = df['new_value'].astype('float64')
df.value.describe()

count    267234.000000
mean        549.850701
std         766.433887
min           0.135000
25%          79.979000
50%         247.030000
75%         695.224000
max        8621.646000
Name: value, dtype: float64

Verifico il valore massimo della feature value che corrisponde al totale della forza lavoro della regione Lombradia nel secondo trimestre del 2019 ed è pari a 8621,646 (valore espresso in migliaia).

In [12]:
df[(df.value == df.value.values.max())]

,itter107,territorio,tipo_dato_fol,tipo dato,sexistat1,sesso,eta1,classe di età,condizione_prof,condizione professionale,condizione_prof_eu,condizione professionale europea,time,seleziona periodo,value,dot_index,new_value
228110,ITC4,Lombardia,POP,popolazione 0 anni e più (in migliaia),9,totale,Y_GE15,15 anni e più,99,totale,TOT,totale,2019-Q2,T2-2019,8621.646,1,8621.646


Infine trasformo i valori in interi.

In [13]:
#Trasformo la feature 'Value' in intero
df['value'] = df['value'] * 1000
df['value'] = df['value'].astype('int')

### Preparazione della condizione professionale
Come dimostrato nella fase esplorativa, sopprimo le righe in cui "condizione_prof_eu" è uguale a "TOT" in quanto ridondanti

In [14]:
# Elimino le osservazioni superflue
df = df[(df['condizione_prof_eu'] == 'TOT')]
#df['condizione professionale'].value_counts()

### Rimozione delle colonne non di interesse
Rimuovo le features non necessarie in quanto ridondanti (espressione della stessa informazione in standard differente) o contenti sempre lo stesso valore, quindi non rilevanti.

In [15]:
df.drop(['tipo_dato_fol', 'tipo dato', 'seleziona periodo', 'dot_index', 'new_value'], axis='columns', inplace=True)
#Elimino anche la feature sexistati1 
df.drop(['sexistat1'], axis='columns', inplace=True)
df.drop(['itter107'], axis='columns', inplace=True)

### Raggruppamento per zone d'Italia
Al fine di poter condurre analisi statistiche per macro zone, raggrupo le regioni nelle 5 zone indivisuate dalla <b>NUTS</b>
(nomenclatura delle unità territoriali statistiche:
1. <b>Nord-ovest:</b>	Valle d'Aosta, Liguria, Lombardia, Piemonte
2. <b>Nord-est:</b>	Trentino-Alto Adige, Veneto, Friuli-Venezia Giulia, Emilia-Romagna
3. <b>Centro:</b> Toscana, Umbria, Marche, Lazio
4. <b>Sud:</b>	Abruzzo, Molise, Campania, Puglia, Basilicata, Calabria
5. <b>Isole:</b> Sicilia, Sardegna

In [16]:
def zona_italia(regione):
    if (regione == "Sardegna") or (regione == "Sicilia"):
        return 'Isole'
    elif ((regione =='Toscana') or (regione == 'Umbria') or (regione == 'Marche') or (regione == 'Lazio')):
        return 'Centro'
    elif ((regione =='Abruzzo') or (regione == 'Molise') or (regione == 'Campania') or (regione == 'Puglia') or (regione == 'Basilicata') or (regione == 'Calabria')):
        return 'Sud'
    elif ((regione =='Liguria') or (regione == 'Lombardia') or (regione == 'Piemonte') or (regione == "Valle d'Aosta / Vallée d'Aosta")):
        return 'Nord-ovest'
    else:
        return 'Nord-est'
df['zona'] = np.vectorize(zona_italia)(df['territorio'])

<b>[NOTA]</b> Rimuovo le rilevazioni trimestrali

In [17]:
#dataframe privo dei dati trimestrali
df_anni = df[~df['time'].str.contains('Q')]

### Nuove classi di età
Il dataset non è in grado di fornire informazioni dettagliate sulle fascie di età rilevanti per l'osservazione dei fenomeni legati all'occupazione, infatti il principale cluster di popolazione attiva (15 - 64 anni) non è ulteriormente suddiviso. Tuttavia è possibile indagare sulle fasce d'età ai margini della fascia tipica di attività. <br>Ricavo pertanto nuove classi di età per esplorare ai margini dell'età lavorativa principale (15-64 anni). In particolare:<br>
1. <b>"65-74 anni"</b>
2. <b>"over 74"</b>

In [18]:
df_p = df_anni.pivot(index=['territorio','zona','time', 'sesso', 'condizione professionale'], columns=['classe di età'], values='value')
df_p = df_p.fillna(0.0)
df_p['65-74 anni'] = df_p['15-74 anni'] - df_p['15-64 anni']
df_p['over 74'] = df_p['15 anni e più'] - df_p['15-74 anni']
df_stacked = df_p.stack('classe di età').reset_index()

### Features categoriche
Preparo le feature che gestirò come Categoriche:

In [19]:
#Test con l'uso dei dati categorici --> ma Seaborn mostra tutte le categorie, anche quando filtrate..
#df['territorio'] = df['territorio'].astype('category')
#df['sesso'] = df['sesso'].astype('category')
df_stacked['classe di età'] = df_stacked['classe di età'].astype('category')
#df['condizione professionale'] = df['condizione professionale'].astype('category')
#df['time'] = df['time'].astype('category')
df_stacked['classe di età'].cat.categories
#df.time.cat.categories
#df['condizione professionale'].cat.categories

Index(['0-14 anni', '15 anni e più', '15-64 anni', '15-74 anni', '65-74 anni',
       'over 74'],
      dtype='object')

### Reshaping del Data Frame
Al fine di poter lavorare agevolmente, rdinando le features di interesse, risulta necessario raggruppare (pivoting). <br>
Gli indici per il raggruppamento sono: <br> 
<b>Territorio, zona, anno,sesso e classe di età </b>
Pertanto in colonna saranno disponibili tutte le features relative alla <b>condizione professionale</b>.


In [20]:
df_pivot = df_stacked.pivot(index=['territorio','zona','time', 'sesso', 'classe di età'], columns=['condizione professionale'], values=0)
df_pivot = df_pivot.fillna(0.0)
df_pivot.head(10)

condizione professionale                    disoccupati  forze lavoro  \
territorio zona time sesso   classe di età                              
Abruzzo    Sud  1996 femmine 0-14 anni              0.0           0.0   
                             15 anni e più      28068.0      224018.0   
                             15-64 anni         28068.0      221626.0   
                             15-74 anni         28068.0      223951.0   
                             65-74 anni             0.0        2325.0   
                             over 74                0.0          67.0   
                     maschi  0-14 anni              0.0           0.0   
                             15 anni e più      33037.0      310355.0   
                             15-64 anni         32999.0      304747.0   
                             15-74 anni         33037.0      310180.0   

condizione professionale                    inattivi  \
territorio zona time sesso   classe di età             
Abruzzo    Sud  1996 femmine 0-14 anni       91031.0   
                             15 anni e più  325979.0   
                             15-64 anni     196145.0   
                             15-74 anni     269588.0   
                             65-74 anni      73443.0   
                             over 74         56391.0   
                     maschi  0-14 anni       95581.0   
                             15 anni e più  201905.0   
                             15-64 anni     108199.0   
                             15-74 anni     165533.0   

condizione professionale                    non cercano e non disponibili a lavorare  \
territorio zona time sesso   classe di età                                             
Abruzzo    Sud  1996 femmine 0-14 anni                                           0.0   
                             15 anni e più                                       0.0   
                             15-64 anni                                          0.0   
                             15-74 anni                                          0.0   
                             65-74 anni                                          0.0   
                             over 74                                             0.0   
                     maschi  0-14 anni                                           0.0   
                             15 anni e più                                       0.0   
                             15-64 anni                                          0.0   
                             15-74 anni                                          0.0   

condizione professionale                    occupati    totale  \
territorio zona time sesso   classe di età                       
Abruzzo    Sud  1996 femmine 0-14 anni           0.0   91031.0   
                             15 anni e più  195950.0  549996.0   
                             15-64 anni     193559.0  417771.0   
                             15-74 anni     195884.0  493539.0   
                             65-74 anni       2325.0   75768.0   
                             over 74            66.0   56457.0   
                     maschi  0-14 anni           0.0   95581.0   
                             15 anni e più  277318.0  512259.0   
                             15-64 anni     271748.0  412946.0   
                             15-74 anni     277143.0  475713.0   

condizione professionale                    zona grigia dell'inattività  
territorio zona time sesso   classe di età                               
Abruzzo    Sud  1996 femmine 0-14 anni                              0.0  
                             15 anni e più                          0.0  
                             15-64 anni                             0.0  
                             15-74 anni                             0.0  
                             65-74 anni                             0.0  
                             over 74                                0.0  
        

### Calcolo ulteriori features relative ai tassi
Per comodità di analisi, calcolo nuove features: <br><b>Tasso di occupazione:</b> rapporto tra occupati e popolazione (per fascia di età e sesso); <br><b>Tasso di disoccupazione:</b> rapporto tra disoccupati e forza lavoro (per fascia di età e sesso); <br><b>Tasso di attività:</b> rapporto tra forza lavoro e popolazione (per fascia di età e sesso); <br><b>Tasso di inattività:</b> rapporto tra inattivi e popolazione (per fascia di età e sesso);<br><b>Gap Occupazionale:</b> Differenza tra tasso di occupazione e tasso di attività <br> Preparo inoltre le relative etichette che saranno comode nella visualizzazione.

In [21]:
df_pivot['tasso_occupazione'] = (df_pivot['occupati'] / df_pivot['totale']) * 100
df_pivot['tasso_disoccupazione'] = (df_pivot['disoccupati'] / df_pivot['forze lavoro']) * 100
df_pivot['tasso_attività'] = (df_pivot['forze lavoro'] / df_pivot['totale']) * 100
df_pivot['tasso_inattività'] = (df_pivot['inattivi'] / df_pivot['totale']) * 100
df_pivot['gap_occupazionale']=  df_pivot['tasso_attività'] - df_pivot['tasso_occupazione']
#Definisco le atichette per la visualizzazione
labels_tassi = {"tasso_occupazione" : "Tasso di occupazione", "tasso_disoccupazione" : "Tasso di disoccupazione",
         "tasso_attività": "Tasso di attvità", "tasso_inattività" : "Tasso di inattività", 
          "gap_occupazionale" : "Gap occupazionale"}
df_pivot = df_pivot.fillna(0.0)
df_pivot.head()

condizione professionale                    disoccupati  forze lavoro  \
territorio zona time sesso   classe di età                              
Abruzzo    Sud  1996 femmine 0-14 anni              0.0           0.0   
                             15 anni e più      28068.0      224018.0   
                             15-64 anni         28068.0      221626.0   
                             15-74 anni         28068.0      223951.0   
                             65-74 anni             0.0        2325.0   

condizione professionale                    inattivi  \
territorio zona time sesso   classe di età             
Abruzzo    Sud  1996 femmine 0-14 anni       91031.0   
                             15 anni e più  325979.0   
                             15-64 anni     196145.0   
                             15-74 anni     269588.0   
                             65-74 anni      73443.0   

condizione professionale                    non cercano e non disponibili a lavorare  \
territorio zona time sesso   classe di età                                             
Abruzzo    Sud  1996 femmine 0-14 anni                                           0.0   
                             15 anni e più                                       0.0   
                             15-64 anni                                          0.0   
                             15-74 anni                                          0.0   
                             65-74 anni                                          0.0   

condizione professionale                    occupati    totale  \
territorio zona time sesso   classe di età                       
Abruzzo    Sud  1996 femmine 0-14 anni           0.0   91031.0   
                             15 anni e più  195950.0  549996.0   
                             15-64 anni     193559.0  417771.0   
                             15-74 anni     195884.0  493539.0   
                             65-74 anni       2325.0   75768.0   

condizione professionale                    zona grigia dell'inattività  \
territorio zona time sesso   classe di età                                
Abruzzo    Sud  1996 femmine 0-14 anni                              0.0   
                             15 anni e più                          0.0   
                             15-64 anni                             0.0   
                             15-74 anni                             0.0   
                             65-74 anni                             0.0   

condizione professionale                    tasso_occupazione  \
territorio zona time sesso   classe di età                      
Abruzzo    Sud  1996 femmine 0-14 anni               0.000000   
                             15 anni e più          35.627532   
                             15-64 anni             46.331363   
                             15-74 anni             39.689670   
                             65-74 anni              3.068578   

condizione professionale                    tasso_disoccupazione  \
territorio zona time sesso   classe di età                         
Abruzzo    Sud  1996 femmine 0-14 anni                  0.000000   
                             15 anni e più             12.529350   
                             15-64 anni                12.664579   
                             15-74 anni                12.533099   
                             65-74 anni                 0.000000   

condizione professionale                    tasso_attività  tasso_inattività  \
territorio zona time sesso   classe di età                                     
Abruzzo    Sud  1996 femmine 0-14 anni            0.000000        100.000000   
                             15 anni e più       40.730842         59.269340   
                             15-64 anni          53.049637         46.950363   
                             15-74 anni          45.376556         54.623444   
                             65-74 anni           3.068578         96.931422

## Data Visualization
### Andamento tasso di occupazione
Trattandosi di una serie temporale di rilevazioni, la prima visualizzazione significativa che intendo condurre è relativa all'andamento dei diversi tassi nel tempo, mettendo in evidenza le realzioni tra di essi e il confronto tra regioni e zone d'italia.<br>
Altro elemento da tenere in considerazione è la classe di età che per le analisi principali può essere fissata a "16-64", in quanto risulta essere la fascia di attività "standard" per l'età lavorativa.<br>
La prima visualizzazione mostra l'andamento del tasso di occupazione dal 1996 al 2018, ogni linea rappresenta una regione, il raggruppamento per colore indica la zona d'Italia.<br>
(Modalità interattiva - Plotly Express)

In [23]:
df1 = df_pivot[(((df_pivot.index.get_level_values(4) == '15-64 anni')) & ((df_pivot.index.get_level_values(3)) == 'totale'))]
fig = px.line(df1, x=df1.index.get_level_values(2), y='tasso_occupazione',
               title='Andamento Tasso di occupazione in italia 15 - 64 anni (per zone)', line_group= df1.index.get_level_values(0), 
              hover_name= df1.index.get_level_values(0), color=df1.index.get_level_values(1),
              category_orders={"color": df1.sort_values(by='tasso_occupazione', ascending = False).index.get_level_values(1).to_list()},
                          labels={ # replaces default labels by column name
                              "color": "Zona d'Italia",  "x": "Anno di riferimento", "tasso_occupazione": labels_tassi.get("tasso_occupazione") + "(%)"
            })
fig.show()

### Esplorazione completa dei tassi
Analogo al grafico precedente ma consente di navigare tra i diversi tassi e scegliere la classe di età d'interesse.
Oltre alla conferma che la classe di età "standard" è quella tra i 14 e 64 anni, si possono apprezzare alcune particolarità regionali sul tasso di attività/inattività anche nelle fasce 65-74. <br>
Uno degli elementi più significatifi riguarda l'andamento della disoccuapazione che tra il 2006 e il 2011 ha raggiunto valori minimi, pertanto è interessante analizzare i fenomeni correlati durante il periodo, e nei periodi precedenti e successivo. <br>

(Modalità interattiva Dash - Plotly Express)

In [25]:
df2 = df_pivot[((df_pivot.index.get_level_values(3)) == 'totale')]

classi_età = df2.index.get_level_values(4).unique()

app = dash.Dash(__name__)


app.layout = html.Div([
    html.P("Esplorazione dei tassi regionali (per zone)",
           style={"font-family":"arial", 'font-size': '2rem','font-style': 'italic', 'text-align': 'center'}
          ),
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in df2.columns[7:]],
        value=df2.columns[7],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px darkgrey solid'},
    ),
        
    html.P(" "),
    dcc.RadioItems(
        id="checklist",
        options=[{"label": x, "value": x} 
                 for x in classi_età],
        value="15-64 anni",
        labelStyle={'display': 'inline-block'},
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px darkgrey solid'}
    ),
     
    dcc.Graph(id="time-series-chart"),
])


@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")],
    [Input("checklist", "value")])
def display_time_series(ticker, età):
    df3 = df2[((df2.index.get_level_values(4)) == età)]
    fig = px.line(df3, x=df3.index.get_level_values(2), y=ticker,
                  line_group= df3.index.get_level_values(0), 
                  hover_name= df3.index.get_level_values(0), 
                  color=df3.index.get_level_values(1),
                  category_orders={"color": df3.sort_values(by=ticker, ascending = False).index.get_level_values(1).to_list()},
                  labels={ # replaces default labels by column name
                    "color": "Zona d'Italia",  "x": "Anno di riferimento", ticker : labels_tassi.get(ticker) + " (%)"
                  })
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### Confronto tassi su base regionale
In questa visualizzazione tutti i tassi per regione vengono rappresentati conntemporaneamente. <br>
E' interessante confrontare il diverso andamento tra le regioni del nord e quelle del meridione, in particolare si noti l'andamento del tasso di inattività nel periodo di minor disoccupazione (2006-2011): in alcune regioni del sud (es. Campania e Calabria) si evidenzia un aumento del tasso di inattività nello stesso periodo, mentre nelle regioni del Nord questo non accade. <br>
Si nota che in alcune regioni del Sud l'abbassamento del tasso di disoccupazione non coincide con l'aumento del tasso di occupazione, che invece decresce a causa dell'aumento degli inattivi.<br>
(Modalità interattiva Dash - Plotly Express)

In [27]:
regioni = df1.index.get_level_values(0).unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in regioni],
        value=regioni[0],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px darkgrey solid'},
    ),
    
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    df3 = df1[((df1.index.get_level_values(0)) == ticker)]
    fig = px.line(df3, x=df3.index.get_level_values(2), y=df3.columns[7:],
                  #line_group= df1.index.get_level_values(0), 
                  #hover_name= df3.columns[7:].to_list(), 
                  #color=df3.columns[7:],
                  category_orders={"variable": df3[(df3.columns[7:])].max().sort_values(ascending = False).index.to_list()},
                  labels={ # replaces default labels by column name
                    "variable": "Tasso",  "x": "Anno di riferimento", "value" : "Tassi"
                  #},
                  #color_discrete_map={ # replaces default color mapping by value
                #"Sud": "crimson", "Nord": "darkblue", "Isole":'Darkcyan', "Centro":'DarkGoldenrod'
                  })
    #fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(title='Andamento dei Tassi per regione')
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Ancora, i tassi regionali sono confrontati con la media dei tassi della zona geografica di riferimento, in questa visualizzazione si evidenzia come alcune regioni presentino indicatori "sopra" la media della zona geograficica (Abruzzo, Sardegna,...), o siano in linea (Basilicata, Piemonte,...) o sotto di essa (Calabria, Campania, Sicilia, ...). <br>
(Modalità interattiva Dash - Plotly)

In [29]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in regioni],
        value=regioni[0],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px darkgrey solid'},
    ),
    
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    df3 = df1[((df1.index.get_level_values(0)) == ticker)]
    df4 = df1[((df1.index.get_level_values(1))==df3.index.get_level_values(1)[1])]
    df4 = df4.groupby(by=df4.index.get_level_values(2)).mean()
    fig = go.Figure()
    fig = make_subplots(rows=4, cols=1, vertical_spacing=0.1, row_heights=[500,500,500,500],
                        #shared_xaxes=True,
                        subplot_titles=("Tasso di occupazione", "Tasso di disoccupazione", "Tasso di attività", "Gap occupazionale")
                       )
    
    # Create and style traces, dash options include 'dash', 'dot', and 'dashdot'
        
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['tasso_occupazione'], name='Tasso di occupazione ' + ticker,
                         text=ticker, line=dict(color='Darkseagreen', width=4)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df4.index.get_level_values(0), y=df4['tasso_occupazione'], name='Tasso di occupazione zona ' + df3.index.get_level_values(1)[1],
                         text=df3.index.get_level_values(1)[1],line=dict(color='Darkseagreen', width=3,dash='dashdot')), row=1, col=1)
    #fig.update_xaxes(title_text="Anno di riferimento", row=1, col=1)
    fig.update_yaxes(title_text="Tasso di occupazione (%)",row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['tasso_disoccupazione'], name = 'Tasso di disoccupazione ' + ticker,
                         text=ticker, line=dict(color='crimson', width=4)),row=2, col=1)
    fig.add_trace(go.Scatter(x=df4.index.get_level_values(0), y=df4['tasso_disoccupazione'], name = 'Tasso di disoccupazione zona ' + df3.index.get_level_values(1)[1],
                         text=df3.index.get_level_values(1)[1], line=dict(color='crimson', width=3,dash='dashdot')),row=2, col=1)
    #fig.update_xaxes(title_text="Anno di riferimento", row=2, col=1)
    fig.update_yaxes(title_text="Tasso di disoccupazione (%)",row=2, col=1)   
    
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['tasso_attività'], name = 'Tasso di attività ' + ticker,
                         text=ticker, line=dict(color='goldenrod', width=4)),row=3, col=1)
    fig.add_trace(go.Scatter(x=df4.index.get_level_values(0), y=df4['tasso_attività'], name = 'Tasso di attvità zona ' + df3.index.get_level_values(1)[1],
                         text=df3.index.get_level_values(1)[1], line=dict(color='goldenrod', width=3,dash='dashdot')),row=3, col=1)
    #fig.update_xaxes(title_text="Anno di riferimento", row=3, col=1)
    fig.update_yaxes(title_text="Tasso di attività (%)",row=3, col=1)  
    
    fig.add_trace(go.Scatter(x=df3.index.get_level_values(2), y=df3['gap_occupazionale'], name = 'Gap occupazionale ' + ticker,
                         text=ticker, line=dict(color='darkturquoise', width=4)),row=4, col=1)
    fig.add_trace(go.Scatter(x=df4.index.get_level_values(0), y=df4['gap_occupazionale'], name = 'Gap occupazionale zona ' + df3.index.get_level_values(1)[1],
                         text=df3.index.get_level_values(1)[1], line=dict(color='darkturquoise', width=3,dash='dashdot')),row=4, col=1)
    #fig.update_xaxes(title_text="Anno di riferimento", row=3, col=1)
    fig.update_yaxes(title_text="Gap occupazionale (%)",row=4, col=1)


    fig.update_layout(height=900, width=1600, title="Confronto tassi con media nazionale per zona di riferimento", hovermode="x unified")
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### Analisi dei tassi per anno
Attraverso la selezione dell'anno (e potenzialmente di un arbitrio periodo temporale), è possibile osservare il rapporto tra tasso di attività e tasso di disoccupazione, questo rapporto mostra nel Boubble Chart le regioni già individuate in precedenza essere raggruppate in basso a sinistra. <br>
Le bolle sono ancora una volta raggruppate per colore = zona d'italia, inoltre la dimensione fornisce indicazioni circa il valore del "Gap Occupazionale".<br>
In basso sono riportati i valori dei tassi per l'anno selezionati, suddivisi per zona d'italia e sesso dei cittadini.<br>
(Modalità interattiva Dash - Plotly)


In [51]:
#per le pie charts
df7 = df_pivot[(((df_pivot.index.get_level_values(4) == '15-64 anni')) & ((df_pivot.index.get_level_values(3)) != 'totale'))]

anni = df1.index.get_level_values(2).unique()
zone = df1.index.get_level_values(1).unique()
regioni = df1.index.get_level_values(0).unique()

sizeref = 2.*df1['gap_occupazionale'].max()/(60.**2)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="ticker",
        options=[{"label": x, "value": x} 
                 for x in anni],
        value=anni[0],
        clearable=False,
        style={"font-family":"arial", 'font-size': '1rem','font-style': 'italic','border':'2px darkgrey solid'},
    ),
    
    dcc.Graph(id="time-series-chart"),
])

@app.callback(
    Output("time-series-chart", "figure"), 
    [Input("ticker", "value")])
def display_time_series(ticker):
    fig = go.Figure()
    fig = make_subplots(rows=2, cols=5, vertical_spacing=0.2, row_heights=[500,500],
                    specs=[[{"type": "xy","colspan": 5}, None, None, None, None],
                          [{"type": "bar"},{"type": "bar"},{"type": "bar"},{"type": "bar"},{"type": "bar"}]],
                    #shared_xaxes=True,
                    subplot_titles=("", "Nord-ovest", "Nord-est", "Centro", "Sud", "Isole"),
                    shared_yaxes=True
                    )

    for zona in zone:
        df6 = df1[(((df1.index.get_level_values(1)) == zona) & ((df1.index.get_level_values(2)) == ticker))]
        fig.add_trace(go.Scatter(
            x=df6['tasso_occupazione'], y=df6['tasso_attività'],
            name=zona, text=df6.index.get_level_values(0),
            marker_size=df6['gap_occupazionale'],
            ), row=1, col=1)
    fig.update_traces(mode='markers', marker=dict(sizemode='area',
                                              sizeref=sizeref, line_width=2))   
    df8 = df7[((df7.index.get_level_values(2)) == ticker)].groupby(level=["zona","sesso"]).mean()

    for index, zona in enumerate(zone):
        if index == 0 :
            fig.add_trace(
               go.Bar(
                    x=df8.columns[7:],
                    y=df8.xs(zona).T['maschi'][7:],
                    name='Maschi',
                    hovertext = "Maschi",
                    legendgroup ="maschi",
                    marker_color='darkgrey'),
                2, index+1)
            fig.add_trace(
                go.Bar(
                    x=df8.columns[7:],
                    y=df8.xs(zona).T['femmine'][7:],
                    name='Femmine',
                    legendgroup = "femmine",
                    hovertext = "Femmine",
                    marker_color='indianred'),
                2, index+1)
        else:
            fig.add_trace(
               go.Bar(
                    x=df8.columns[7:],
                    y=df8.xs(zona).T['maschi'][7:],
                    name='Maschi',
                    showlegend=False,
                    legendgroup = "maschi",
                    hovertext = "Maschi",
                    marker_color='darkgrey'),
                2, index+1)
            fig.add_trace(
                go.Bar(
                    x=df8.columns[7:],
                    y=df8.xs(zona).T['femmine'][7:],
                    name='Femmine',
                    legendgroup = "femmine",
                    showlegend=False,
                    hovertext='Femmine',
                    marker_color='indianred'),
                2, index+1)
    fig.update_layout(
    height=900, width=1600,
    title='Tasso di attività vs tasso di occupazione, ' + ticker,
    legend= {'itemsizing': 'constant'},
    xaxis=dict(
        title='Tasso di occupazione (%)',
        gridcolor='white',
        type='log',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Tasso di attività (%)',
        gridcolor='white',
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    )
    return fig

app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run